In [1]:
import os, argparse, json, re
from collections import defaultdict
import customutils
import pandas as pd
import numpy as np

LABELS = customutils.get_classes('configs/halpe26_classes.txt')
LABELS_COCO = customutils.get_classes('configs/coco_classes.txt')
LABELS_FOOT = ['LBigToe','RBigToe','LSmallToe','RSmallToe','LHeel','RHeel']

df_running_annotations = customutils.load_images_dataframe()
df_running_annotations

,image_id,pose_id,file_name,frame,person
0,1000,1000,Athletics_Mixed_Tokyo_2020_20_1.mp4,0,0
1,1001,1001,Athletics_Mixed_Tokyo_2020_20_1.mp4,1,0
2,1002,1002,Athletics_Mixed_Tokyo_2020_20_1.mp4,2,0
3,1003,1003,Athletics_Mixed_Tokyo_2020_20_1.mp4,3,0
4,1004,1004,Athletics_Mixed_Tokyo_2020_20_1.mp4,4,0
...,...,...,...,...,...
12105,45160,145160,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,160,1
12106,45161,145161,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,161,1
12107,45162,145162,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,162,1
12108,45163,145163,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,163,1


In [2]:
df = customutils.load_keypoints_dataframe()
df[df.duplicated(subset=['id'],keep=False)]

,id,image_id,category_id,bbox,area,iscrowd,num_keypoints,keypoints


In [3]:

df = df.drop(columns=['iscrowd', 'num_keypoints'])
df = df.set_index(['id'])
df

,image_id,category_id,bbox,area,keypoints
id,,,,,
1000,1000,1,"[957.5031, 204.6478, 294.4987, 641.9175]",189043.869257,"[1132.7201, 271.7583, 1, 1151.8834, 256.5806, ..."
1001,1001,1,"[895.2875, 201.4572, 396.59609999999986, 639.6...",253699.828317,"[1135.4403, 262.4473, 1, 1151.8834, 253.39, 1,..."
1002,1002,1,"[876.5417, 207.8383, 455.8919000000001, 641.2885]",292358.232713,"[1132.7201, 255.8056, 1, 1148.6928, 247.0089, ..."
1003,1003,1,"[892.6316, 187.0998, 445.91919999999993, 658.8...",293787.800419,"[1129.5295, 249.4245, 1, 1145.5023, 240.6278, ..."
1004,1004,1,"[912.8355, 201.4572, 406.572, 681.1703]",276944.771212,"[1132.7201, 249.4245, 1, 1148.6928, 240.6278, ..."
...,...,...,...,...,...
145160,45160,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,..."
145161,45161,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,..."
145162,45162,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,..."


In [4]:

df_categories = customutils.load_categories_dataframe()
kpts_classes = df_categories[df_categories['id'] == 1]['keypoints'][0]
kpts_classes

['nose',
 'left_eye',
 'right_eye',
 'left_ear',
 'right_ear',
 'left_shoulder',
 'right_shoulder',
 'left_elbow',
 'right_elbow',
 'left_wrist',
 'right_wrist',
 'left_hip',
 'right_hip',
 'left_knee',
 'right_knee',
 'left_ankle',
 'right_ankle',
 'head',
 'neck',
 'hip',
 'left_big_toe',
 'right_big_toe',
 'left_small_toe',
 'right_small_toe',
 'left_heel',
 'right_heel']

In [5]:

df['dist_alphapose'] = np.ones((len(df),1)) * np.inf
df['dist_openpose'] = np.ones((len(df),1)) * np.inf
df['dist_vitpose'] = np.ones((len(df),1)) * np.inf
df['dist_hrnet'] = np.ones((len(df),1)) * np.inf
df['conf_alphapose'] = float(0.0)
df['conf_openpose'] = float(0.0)
df['conf_vitpose'] = float(0.0)
df['conf_hrnet'] = float(0.0)
df['oks_alphapose'] = float(0.0)
df['oks_openpose'] = float(0.0)
df['oks_vitpose'] = float(0.0)
df['oks_hrnet'] = float(0.0)
df['oks_coco_alphapose'] = float(0.0)
df['oks_coco_openpose'] = float(0.0)
df['oks_coco_vitpose'] = float(0.0)
df['oks_coco_hrnet'] = float(0.0)
df['oks_coco_cotracker3'] = float(0.0)
df['oks_foot_alphapose'] = float(0.0)
df['oks_foot_openpose'] = float(0.0)
df['oks_foot_vitpose'] = float(0.0)
df['oks_foot_hrnet'] = float(0.0)
df['kpts_alphapose'] = ""#np.zeros((len(df),78)).tolist()
df['kpts_openpose'] = ""#np.zeros((len(df),78)).tolist()
df['kpts_vitpose'] = ""#np.zeros((len(df),78)).tolist()
df['kpts_hrnet'] = ""#np.zeros((len(df),78)).tolist()
df

,image_id,category_id,bbox,area,keypoints,dist_alphapose,dist_openpose,dist_vitpose,dist_hrnet,conf_alphapose,...,oks_coco_hrnet,oks_coco_cotracker3,oks_foot_alphapose,oks_foot_openpose,oks_foot_vitpose,oks_foot_hrnet,kpts_alphapose,kpts_openpose,kpts_vitpose,kpts_hrnet
id,,,,,,,,,,,,,,,,,,,,,
1000,1000,1,"[957.5031, 204.6478, 294.4987, 641.9175]",189043.869257,"[1132.7201, 271.7583, 1, 1151.8834, 256.5806, ...",inf,inf,inf,inf,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,,,,
1001,1001,1,"[895.2875, 201.4572, 396.59609999999986, 639.6...",253699.828317,"[1135.4403, 262.4473, 1, 1151.8834, 253.39, 1,...",inf,inf,inf,inf,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,,,,
1002,1002,1,"[876.5417, 207.8383, 455.8919000000001, 641.2885]",292358.232713,"[1132.7201, 255.8056, 1, 1148.6928, 247.0089, ...",inf,inf,inf,inf,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,,,,
1003,1003,1,"[892.6316, 187.0998, 445.91919999999993, 658.8...",293787.800419,"[1129.5295, 249.4245, 1, 1145.5023, 240.6278, ...",inf,inf,inf,inf,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,,,,
1004,1004,1,"[912.8355, 201.4572, 406.572, 681.1703]",276944.771212,"[1132.7201, 249.4245, 1, 1148.6928, 240.6278, ...",inf,inf,inf,inf,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145160,45160,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",inf,inf,inf,inf,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,,,,
145161,45161,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",inf,inf,inf,inf,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,,,,
145162,45162,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",inf,inf,inf,inf,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,,,,


In [6]:

def edit_keypoints(kpts):
    kpts = np.array(kpts).reshape(-1,3).astype(float)
    vi = kpts[:,2]
    kpts = kpts[:,0:2]
    return kpts, vi

def get_keypoint(kpts, id):
    index = kpts_classes.index(id)
    return [kpts[index*3],kpts[(index*3)+1]]

def load_keypoints(root, model):
    dist = 'dist_' + str.lower(model)
    kpts = 'kpts_' + str.lower(model)
    conf = 'conf_' + str.lower(model)
    oks = 'oks_' + str.lower(model)
    oks_coco = 'oks_coco_' + str.lower(model)
    oks_foot = 'oks_foot_' + str.lower(model)
    file = os.path.join(root, model, 'person_keypoints_running.json')
    with open(file, 'r') as f:
        predictions = json.load(f)
        d = {}
        
        for prediction in predictions:
            # if prediction['image_id'] != 46162:
            #     continue
            pose_id = -1
            mask_frame = df['image_id'] == prediction['image_id']
            gts = df[mask_frame]
            dt = prediction['keypoints']
            kpts1, vi1 = edit_keypoints(dt)
            min_dist = np.inf #np.ones((len(kpts1),1)) * 
            
            # for i in range(len(gts)):
            for pose_id in gts.index:
                gt = gts['keypoints'][pose_id]
                kpts2, vi2 = edit_keypoints(gt)
                d = np.linalg.norm(kpts1 - kpts2, ord=2, axis=1)
                v = np.ones(len(d))

                for part in range(len(d)):
                    if vi1[part] == 0 or vi2[part] == 0:
                        d[part] = 0
                        v[part] = 0
                                        
                area = customutils.compute_area_keypoints(gt)

                # compute head size for distance normalization
                head = get_keypoint(gt,"head")
                neck = get_keypoint(gt,"neck")

                headSize = 1
                if (len(head) > 0 and len(neck) > 0):
                    headSize = customutils.get_head_size(head[0], head[1], neck[0], neck[1])
                # normalize distance
                dNorm = np.sum(d)/headSize

                oks_val = customutils.OKS(gt, dt, area, 0, 26)
                oks_val_coco = customutils.OKS(gt, dt, area, 0, 17)
                oks_val_foot = customutils.OKS(gt, dt, area, 20, 26)

                if dNorm < min_dist and dNorm < gts[dist][pose_id]:
                    min_dist = dNorm
                    df.loc[pose_id, dist] = min_dist
                    df.loc[pose_id, oks] = oks_val
                    df.loc[pose_id, oks_coco] = oks_val_coco
                    df.loc[pose_id, oks_foot] = oks_val_foot
                    df.loc[pose_id, kpts] = ','.join(str(element) for element in dt)
                    df.loc[pose_id, conf] = prediction['score']
            
        # df_running_annotations['score_' + str.lower(model)] = df_running_annotations['image_id'].map(d)

In [7]:
root = '../videos/results/'
load_keypoints(root, 'openpose')
load_keypoints(root, 'alphapose')
load_keypoints(root, 'ViTPose')
load_keypoints(root, 'HRNet')

In [8]:
def computeAP(oks, df, thresh):
    T = len(df[oks])
    idxs = np.argwhere(df[oks] >= thresh)
    ap = 100.0*len(idxs)/T
    return ap

def computeAllAP(oks, df):
    distThresh = 0.5
    ap50 = computeAP(oks, df, distThresh)
    ap = np.copy(ap50)
    for i in range(9):
        distThresh += 0.05
        ap += computeAP(oks, df, distThresh)
    ap75 = computeAP(oks, df, 0.75)
    ap = ap/10
    print('AP (' + oks + '): ' + str(ap))
    print('AP0.5 (' + oks + '): ' + str(ap50))
    print('AP0.75 (' + oks + '): ' + str(ap75))


In [9]:
# remove unmatched poses/frames
df_clean = df.loc[(df['conf_openpose'] > float(0)) & (df['conf_alphapose'] > float(0)) & (df['conf_vitpose'] > float(0)) & (df['conf_hrnet'] > float(0))]
df_clean

,image_id,category_id,bbox,area,keypoints,dist_alphapose,dist_openpose,dist_vitpose,dist_hrnet,conf_alphapose,...,oks_coco_hrnet,oks_coco_cotracker3,oks_foot_alphapose,oks_foot_openpose,oks_foot_vitpose,oks_foot_hrnet,kpts_alphapose,kpts_openpose,kpts_vitpose,kpts_hrnet
id,,,,,,,,,,,,,,,,,,,,,
1000,1000,1,"[957.5031, 204.6478, 294.4987, 641.9175]",189043.869257,"[1132.7201, 271.7583, 1, 1151.8834, 256.5806, ...",7.334499,7.013446,6.448374,6.367255,0.877560,...,0.837240,0.0,0.734421,0.742980,0.843515,0.846251,"1136.07421875,266.266845703125,2,1155.60852050...","1139.55,268.183,2,1160.02,256.521,2,1139.39,24...","1135.80859375,267.2095031738281,2,1154.4865722...","1135.575439453125,264.9145202636719,2,1153.360..."
1001,1001,1,"[895.2875, 201.4572, 396.59609999999986, 639.6...",253699.828317,"[1135.4403, 262.4473, 1, 1151.8834, 253.39, 1,...",6.483823,7.133457,5.973131,8.286392,0.867543,...,0.848443,0.0,0.967516,0.874478,0.882239,0.805016,"1134.0335693359375,257.7659912109375,2,1153.15...","1139.57,259.547,2,1157.16,247.864,2,1139.4,244...","1137.83642578125,258.818115234375,2,1154.47412...","1133.6328125,259.5325622558594,2,1151.58813476..."
1002,1002,1,"[876.5417, 207.8383, 455.8919000000001, 641.2885]",292358.232713,"[1132.7201, 255.8056, 1, 1148.6928, 247.0089, ...",6.342289,5.756402,4.706753,4.713285,0.869874,...,0.910131,0.0,0.978979,0.959724,0.976162,0.957351,"1128.94189453125,252.69618225097656,2,1148.562...","1139.6,247.833,2,1157.25,244.735,2,1139.4,235....","1139.1051025390625,250.20867919921875,2,1152.7...","1136.784423828125,251.25259399414062,2,1154.65..."
1003,1003,1,"[892.6316, 187.0998, 445.91919999999993, 658.8...",293787.800419,"[1129.5295, 249.4245, 1, 1145.5023, 240.6278, ...",4.635418,5.450903,3.991620,4.525670,0.887327,...,0.916758,0.0,0.989497,0.975538,0.980320,0.971877,"1129.5250244140625,242.39736938476562,2,1143.6...","1136.53,244.883,2,1148.4,241.765,2,1133.67,227...","1127.6134033203125,245.468505859375,2,1148.882...","1132.8712158203125,246.05426025390625,2,1151.2..."
1004,1004,1,"[912.8355, 201.4572, 406.572, 681.1703]",276944.771212,"[1132.7201, 249.4245, 1, 1148.6928, 240.6278, ...",5.010565,5.075978,5.594584,5.901880,0.891673,...,0.943026,0.0,0.968040,0.950898,0.901227,0.852091,"1127.0386962890625,248.37823486328125,2,1148.4...","1133.7,244.898,2,1148.37,241.799,2,1133.6,227....","1127.815673828125,244.443359375,2,1145.7788085...","1129.1875,247.21041870117188,2,1149.0015869140..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145159,45159,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",11.914365,9.199873,6.092382,8.565661,0.842258,...,0.670067,0.0,0.675285,0.784656,0.928922,0.880220,"822.9765625,304.3045959472656,2,839.5335693359...","833.28,294.867,2,845.198,283.013,2,812.954,280...","830.2598876953125,319.13177490234375,2,846.903...","831.77978515625,319.06134033203125,2,846.67651..."
145160,45160,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",11.937775,8.640361,5.988541,7.480348,0.836475,...,0.731545,0.0,0.684083,0.793617,0.905955,0.874065,"824.4197387695312,305.15618896484375,2,840.958...","833.336,294.917,2,845.255,283.051,2,812.973,28...","831.824951171875,318.6900634765625,2,840.14489...","827.8294677734375,318.771484375,2,843.38122558..."
145161,45161,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",11.866606,8.495965,5.262168,8.053429,0.848611,...,0.683649,0.0,0.654433,0.802060,0.933736,0.883098,"825.8770751953125,304.82806396484375,2,842.422...","833.278,297.716,2,845.184,285.971,2,812.94,283...","829.6857299804688,318.8739013671875,2,838.0119...","831.06982421875,319.2327880859375,2,845.689453..."


In [10]:
computeAllAP('oks_openpose', df_clean)
computeAllAP('oks_alphapose', df_clean)
computeAllAP('oks_vitpose', df_clean)
computeAllAP('oks_hrnet', df_clean)
computeAllAP('oks_coco_openpose', df_clean)
computeAllAP('oks_coco_alphapose', df_clean)
computeAllAP('oks_coco_vitpose', df_clean)
computeAllAP('oks_coco_hrnet', df_clean)
computeAllAP('oks_foot_openpose', df_clean)
computeAllAP('oks_foot_alphapose', df_clean)
computeAllAP('oks_foot_vitpose', df_clean)
computeAllAP('oks_foot_hrnet', df_clean)

AP (oks_openpose): 50.67796610169491
AP0.5 (oks_openpose): 87.28400165357586
AP0.75 (oks_openpose): 53.50144687887557
AP (oks_alphapose): 54.22819346837535
AP0.5 (oks_alphapose): 92.85655229433651
AP0.75 (oks_alphapose): 55.77511368334022
AP (oks_vitpose): 67.713104588673
AP0.5 (oks_vitpose): 94.69202149648615
AP0.75 (oks_vitpose): 79.51219512195122
AP (oks_hrnet): 58.32410086812733
AP0.5 (oks_hrnet): 93.37742868954113
AP0.75 (oks_hrnet): 62.80281107895825
AP (oks_coco_openpose): 60.48284415047541
AP0.5 (oks_coco_openpose): 89.71475816453079
AP0.75 (oks_coco_openpose): 68.92930963207937
AP (oks_coco_alphapose): 60.149648615130204
AP0.5 (oks_coco_alphapose): 94.60107482430756
AP0.75 (oks_coco_alphapose): 68.82182720132286
AP (oks_coco_vitpose): 69.93716411740388
AP0.5 (oks_coco_vitpose): 95.60148821827201
AP0.75 (oks_coco_vitpose): 84.99379909053327
AP (oks_coco_hrnet): 66.54485324514262
AP0.5 (oks_coco_hrnet): 95.60975609756098
AP0.75 (oks_coco_hrnet): 79.35510541546094
AP (oks_foot_op

In [11]:
df[['image_id','oks_alphapose','oks_openpose','oks_vitpose',	'oks_coco_alphapose',	'oks_coco_openpose',	'oks_coco_vitpose',	'oks_foot_alphapose',	'oks_foot_openpose',	'oks_foot_vitpose']]
df2 = df[['oks_alphapose','oks_openpose','oks_vitpose',	'oks_coco_alphapose',	'oks_coco_openpose',	'oks_coco_vitpose',	'oks_foot_alphapose',	'oks_foot_openpose',	'oks_foot_vitpose']].mean()
df2

oks_alphapose         0.730306
oks_openpose          0.688974
oks_vitpose           0.799063
oks_coco_alphapose    0.761487
oks_coco_openpose     0.739856
oks_coco_vitpose      0.811909
oks_foot_alphapose    0.659849
oks_foot_openpose     0.604849
oks_foot_vitpose      0.762665
dtype: float64

In [12]:
df_clean[['image_id','oks_alphapose','oks_openpose','oks_vitpose',	'oks_coco_alphapose',	'oks_coco_openpose',	'oks_coco_vitpose',	'oks_foot_alphapose',	'oks_foot_openpose',	'oks_foot_vitpose']]
df3 = df_clean[['oks_alphapose','oks_openpose','oks_vitpose',	'oks_coco_alphapose',	'oks_coco_openpose',	'oks_coco_vitpose',	'oks_foot_alphapose',	'oks_foot_openpose',	'oks_foot_vitpose']].mean()
df3

oks_alphapose         0.731212
oks_openpose          0.689724
oks_vitpose           0.799940
oks_coco_alphapose    0.762431
oks_coco_openpose     0.740655
oks_coco_vitpose      0.812787
oks_foot_alphapose    0.660667
oks_foot_openpose     0.605559
oks_foot_vitpose      0.763543
dtype: float64

In [13]:
def load_results(root, model):
    file = os.path.join(root, 'results_pckh_' + model + '.json')
    with open(file, 'r') as f:
        predictions = json.load(f)
        d = {}
        for prediction in predictions:
            d[prediction['image_id']] = prediction['score']
            
        df_running_annotations['score_' + str.lower(model)] = df_running_annotations['image_id'].map(d)

In [14]:
# remove unmatched poses/frames
df_excl_hidden = df.loc[(df['oks_foot_alphapose'] > 0.01) | (df['oks_foot_openpose'] > 0.01) | (df['oks_foot_vitpose'] > 0.01)]
df_excl_hidden

,image_id,category_id,bbox,area,keypoints,dist_alphapose,dist_openpose,dist_vitpose,dist_hrnet,conf_alphapose,...,oks_coco_hrnet,oks_coco_cotracker3,oks_foot_alphapose,oks_foot_openpose,oks_foot_vitpose,oks_foot_hrnet,kpts_alphapose,kpts_openpose,kpts_vitpose,kpts_hrnet
id,,,,,,,,,,,,,,,,,,,,,
1000,1000,1,"[957.5031, 204.6478, 294.4987, 641.9175]",189043.869257,"[1132.7201, 271.7583, 1, 1151.8834, 256.5806, ...",7.334499,7.013446,6.448374,6.367255,0.877560,...,0.837240,0.0,0.734421,0.742980,0.843515,0.846251,"1136.07421875,266.266845703125,2,1155.60852050...","1139.55,268.183,2,1160.02,256.521,2,1139.39,24...","1135.80859375,267.2095031738281,2,1154.4865722...","1135.575439453125,264.9145202636719,2,1153.360..."
1001,1001,1,"[895.2875, 201.4572, 396.59609999999986, 639.6...",253699.828317,"[1135.4403, 262.4473, 1, 1151.8834, 253.39, 1,...",6.483823,7.133457,5.973131,8.286392,0.867543,...,0.848443,0.0,0.967516,0.874478,0.882239,0.805016,"1134.0335693359375,257.7659912109375,2,1153.15...","1139.57,259.547,2,1157.16,247.864,2,1139.4,244...","1137.83642578125,258.818115234375,2,1154.47412...","1133.6328125,259.5325622558594,2,1151.58813476..."
1002,1002,1,"[876.5417, 207.8383, 455.8919000000001, 641.2885]",292358.232713,"[1132.7201, 255.8056, 1, 1148.6928, 247.0089, ...",6.342289,5.756402,4.706753,4.713285,0.869874,...,0.910131,0.0,0.978979,0.959724,0.976162,0.957351,"1128.94189453125,252.69618225097656,2,1148.562...","1139.6,247.833,2,1157.25,244.735,2,1139.4,235....","1139.1051025390625,250.20867919921875,2,1152.7...","1136.784423828125,251.25259399414062,2,1154.65..."
1003,1003,1,"[892.6316, 187.0998, 445.91919999999993, 658.8...",293787.800419,"[1129.5295, 249.4245, 1, 1145.5023, 240.6278, ...",4.635418,5.450903,3.991620,4.525670,0.887327,...,0.916758,0.0,0.989497,0.975538,0.980320,0.971877,"1129.5250244140625,242.39736938476562,2,1143.6...","1136.53,244.883,2,1148.4,241.765,2,1133.67,227...","1127.6134033203125,245.468505859375,2,1148.882...","1132.8712158203125,246.05426025390625,2,1151.2..."
1004,1004,1,"[912.8355, 201.4572, 406.572, 681.1703]",276944.771212,"[1132.7201, 249.4245, 1, 1148.6928, 240.6278, ...",5.010565,5.075978,5.594584,5.901880,0.891673,...,0.943026,0.0,0.968040,0.950898,0.901227,0.852091,"1127.0386962890625,248.37823486328125,2,1148.4...","1133.7,244.898,2,1148.37,241.799,2,1133.6,227....","1127.815673828125,244.443359375,2,1145.7788085...","1129.1875,247.21041870117188,2,1149.0015869140..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145159,45159,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",11.914365,9.199873,6.092382,8.565661,0.842258,...,0.670067,0.0,0.675285,0.784656,0.928922,0.880220,"822.9765625,304.3045959472656,2,839.5335693359...","833.28,294.867,2,845.198,283.013,2,812.954,280...","830.2598876953125,319.13177490234375,2,846.903...","831.77978515625,319.06134033203125,2,846.67651..."
145160,45160,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",11.937775,8.640361,5.988541,7.480348,0.836475,...,0.731545,0.0,0.684083,0.793617,0.905955,0.874065,"824.4197387695312,305.15618896484375,2,840.958...","833.336,294.917,2,845.255,283.051,2,812.973,28...","831.824951171875,318.6900634765625,2,840.14489...","827.8294677734375,318.771484375,2,843.38122558..."
145161,45161,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",11.866606,8.495965,5.262168,8.053429,0.848611,...,0.683649,0.0,0.654433,0.802060,0.933736,0.883098,"825.8770751953125,304.82806396484375,2,842.422...","833.278,297.716,2,845.184,285.971,2,812.94,283...","829.6857299804688,318.8739013671875,2,838.0119...","831.06982421875,319.2327880859375,2,845.689453..."


In [15]:
df_oks = df_excl_hidden[['image_id','oks_alphapose','oks_openpose','oks_vitpose','oks_foot_alphapose',	'oks_foot_openpose',	'oks_foot_vitpose']]#.sort_values(['oks_foot_alphapose'])

In [16]:
df_excl_hidden_mean = df_excl_hidden[['oks_alphapose','oks_openpose','oks_vitpose',	'oks_coco_alphapose',	'oks_coco_openpose',	'oks_coco_vitpose',	'oks_foot_alphapose',	'oks_foot_openpose',	'oks_foot_vitpose']].mean()
df_excl_hidden_mean

oks_alphapose         0.741389
oks_openpose          0.699013
oks_vitpose           0.811930
oks_coco_alphapose    0.771852
oks_coco_openpose     0.749642
oks_coco_vitpose      0.823326
oks_foot_alphapose    0.674533
oks_foot_openpose     0.618308
oks_foot_vitpose      0.779642
dtype: float64

In [17]:
df_oks[(df_oks['image_id'] >= 28000) & (df_oks['image_id'] <= 28086)]

,image_id,oks_alphapose,oks_openpose,oks_vitpose,oks_foot_alphapose,oks_foot_openpose,oks_foot_vitpose
id,,,,,,,
28000,28000,0.676283,0.521913,0.407385,0.445789,0.245428,0.841560
28001,28001,0.749039,0.629666,0.680009,0.775663,0.144559,0.191524
28002,28002,0.839913,0.794368,0.913504,0.831797,0.499818,0.947618
28003,28003,0.834034,0.797364,0.908769,0.786182,0.477552,0.941784
28004,28004,0.499277,0.639910,0.776364,0.399249,0.041551,0.577144
...,...,...,...,...,...,...,...
28082,28082,0.776653,0.567564,0.910655,0.687890,0.041135,0.926548
28083,28083,0.645295,0.781287,0.810738,0.167708,0.068445,0.492065
28084,28084,0.002766,0.084242,0.872006,0.011902,0.000000,0.700594


In [18]:
df_running_annotations[df_running_annotations['image_id'] == 28000]

,image_id,pose_id,file_name,frame,person
8379,28000,28000,World_Athletics_Men_5000m_Oregon_2022_20.mp4,0,0


In [19]:
df_running_annotations[df_running_annotations['file_name'] == 'World_Athletics_Men_5000m_Oregon_2022_18.mp4']

,image_id,pose_id,file_name,frame,person
8205,27000,27000,World_Athletics_Men_5000m_Oregon_2022_18.mp4,0,0
8206,27001,27001,World_Athletics_Men_5000m_Oregon_2022_18.mp4,1,0
8207,27002,27002,World_Athletics_Men_5000m_Oregon_2022_18.mp4,2,0
8208,27003,27003,World_Athletics_Men_5000m_Oregon_2022_18.mp4,3,0
8209,27004,27004,World_Athletics_Men_5000m_Oregon_2022_18.mp4,4,0
...,...,...,...,...,...
8374,27082,127082,World_Athletics_Men_5000m_Oregon_2022_18.mp4,82,1
8375,27083,127083,World_Athletics_Men_5000m_Oregon_2022_18.mp4,83,1
8376,27084,127084,World_Athletics_Men_5000m_Oregon_2022_18.mp4,84,1
8377,27085,127085,World_Athletics_Men_5000m_Oregon_2022_18.mp4,85,1
